# Projection of 2D ROOT data into 1D slice
Comparison of 1D slices of ROOT and McStas data
## Import libraries

In [ ]:
import uproot
import os
import numpy as np
import matplotlib.pyplot as plt

import dataconfig  # to get paths to data

%matplotlib widget

import ipywidgets

# choice of colormap for 2D plots
colormap = plt.cm.get_cmap('gist_earth')

## Import 2D McStas data

In [ ]:
path_to_mcstas2D_file = os.path.join(dataconfig.data_mcstas, 'monitor_tx_DENEX.dat')

assert os.path.isfile(path_to_mcstas2D_file), \
    'There is an issue with the chosen McStas output datafile'

# Extract shape of output data
# these values will be used to determine how many lines to read one array
with open(path_to_mcstas2D_file, 'r') as file:
    for line in file:
        if "array_2d" in line:
            type_array = line.rstrip()
            start = type_array.find('(') + 1
            end = type_array.find(')', start)
            nx_value, ny_value = map(int, type_array[start:end].split(','))
        if "xylimits" in line:
            xylims = np.array(line.split(':')[1].split()).astype(np.float)

print(f'Limits of x- and y-axis: {xylims}\nNumber of points: nx={nx_value}, ny={ny_value}')

The McStas TOF axis is expressed in milliseconds.

In [ ]:
data2d = np.genfromtxt(path_to_mcstas2D_file, max_rows=ny_value)

#flip data along y axis 
data2d_mcstas = np.flip(data2d, 0)

# define x, y axes (bin-centered)
dx = (xylims[1] - xylims[0]) / float(nx_value)
dy = (xylims[3] - xylims[2]) / float(ny_value)
xaxis_mcstas = np.linspace(xylims[0] + 0.5*dx, xylims[1] - 0.5*dx, nx_value) * 1.0e3 # s -> ms
yaxis_mcstas = np.linspace(xylims[2] + 0.5*dx, xylims[3] - 0.5*dy, nx_value)

In [ ]:
# function to be updated when changing the range of values to calculate the projection
def plots_for_projection_mcstas(*project_boundaries):
    """ Plot 2D and 1D projection for user-specified range of y-values"""
    CS = ax0[0].contourf(xaxis_mcstas, yaxis_mcstas, data2d_mcstas, cmap=colormap)
    ax0[0].hlines(yaxis_mcstas[project_boundaries[0]], 
                 min(xaxis_mcstas), 
                 max(xaxis_mcstas), 
                 'orange')
    ax0[0].hlines(yaxis_mcstas[project_boundaries[1]], 
                 min(xaxis_mcstas), 
                 max(xaxis_mcstas), 
                 'orange')
     
    # calculate projection
    ax0[1].grid()
    line_mcstas, = ax0[1].plot(xaxis_mcstas, 
                              np.sum(data2d_mcstas[project_boundaries[0]:project_boundaries[1],:], axis=0))
    fig0.colorbar(CS, ax=[ax0[0]], location='right')
    fig0.canvas.draw()


# widget to generate 1D projection of mcstas data
fig0, ax0 = plt.subplots(2, 1, constrained_layout=True, sharex=True)
fig0.canvas.header_visible = False
plots_for_projection_mcstas(*(0, len(yaxis_mcstas)-1))

style_textbox = {'description_width': 'initial'}

def update(change):
    ax0[0].clear()
    ax0[1].clear() 
    plots_for_projection_mcstas(*change.new)

set_limits = ipywidgets.IntRangeSlider(
    value=[0, len(yaxis_mcstas)-1],
    min=0, 
    max=len(yaxis_mcstas)-1,
    step=1,
    description='Index boundaries:',
    continuous_update=False,
    readout_format='d',
    style=style_textbox,
    layout = ipywidgets.Layout(width='400px')
)

set_limits.observe(update, 'value')
set_limits

## Import ROOT data

In [ ]:
assert os.path.exists(dataconfig.data_root), 'The path does not exist.'

ROOTfile = 'Spectrum03_DENEX006_1_18-02-05_0000.root'

dir_with_data = 'Meas_3'

data_to_plot = 'H_TOF,X1-X2_User_2D2_dsp_after_run_3'

file_to_open = os.path.join(dataconfig.data_root, ROOTfile)

assert os.path.isfile(file_to_open), 'There is an issue with the file to be opened.'

# Some metadata related to TOF channel for ROOT file
tof_tick = 25e-6 # in milliseconds (25 ns)

In [ ]:
with uproot.open(file_to_open)[dir_with_data] as myFile:
    for key in myFile.keys():
        if 'BoardParam_run' in str(key):
            myObject = myFile[key]
            nb_xbins = myObject.numbins
            
            for i in range(nb_xbins):
                if 'TOF_Time_Channel_Width' in myObject.xlabels[i]:
                    TOF_Time_Channel_Width = myObject.values[i]
                elif 'TOF_Window_Delay_Register' in myObject.xlabels[i]:
                    TOF_Window_Delay_Register = myObject.values[i]
                    
        # 2D contourplot
        if 'TH2' in str(myFile[key]) and data_to_plot in str(myFile[key]):
            # x_max, x_min, y_max, y_min defined from .root file
            x_min_root = myFile[key].xlow
            x_max_root = myFile[key].xhigh
            bins_x_root = myFile[key].xnumbins
            y_min_root = myFile[key].ylow
            y_max_root = myFile[key].yhigh
            bins_y_root = myFile[key].ynumbins

            # create x- and y-axis
            deltax_root = (x_max_root - x_min_root)/(bins_x_root - 1)
            xaxis_root = x_min_root + deltax_root * np.arange(bins_x_root)
            deltay_root = (y_max_root - y_min_root)/(bins_y_root - 1)
            yaxis_root = y_min_root + deltay_root * np.arange(bins_y_root)
            
            # fill 2d matrice with inverted y-axis
            data2d_root = np.flip(myFile[key].values, 1)

In [ ]:
# function to be updated when changing the range of values to calculate the projection
def plots_for_projection(*project_boundaries):
    """ Plot 2D and 1D projection for user-specified range of y-values"""
    CS = ax1[0].contourf(xaxis_root, yaxis_root, data2d_root.transpose(), cmap=colormap) 

    ax1[0].hlines(yaxis_root[project_boundaries[0]], x_min_root, x_max_root, 'orange')
    ax1[0].hlines(yaxis_root[project_boundaries[1]], x_min_root, x_max_root, 'orange')
    
    # find indices to calculate projection
    ax1[1].plot(xaxis_root, np.sum(data2d_root.transpose()[project_boundaries[0]:project_boundaries[1],:], axis=0))
    ax1[1].grid()
    
    fig1.colorbar(CS, ax=[ax1[0]], location='right')
    fig1.canvas.draw()

# widget to generate 1D projection of root data
fig1, ax1 = plt.subplots(2,1, constrained_layout=True, sharex=True)
fig1.canvas.header_visible = False
plots_for_projection(*(0, len(yaxis_root)-1))

style_textbox = {'description_width': 'initial'}

def update(change):
    ax1[0].clear()
    ax1[1].clear()
    plots_for_projection(*change.new)

set_limits = ipywidgets.IntRangeSlider(
    value=[0, len(yaxis_root)-1],
    min=0, 
    max=len(yaxis_root)-1,
    step=1,
    description='Index boundaries:', 
    continuous_update=False,
    readout_format='d',
    style=style_textbox,
    layout = ipywidgets.Layout(width='400px')
)

set_limits.observe(update, 'value')
set_limits

## Creating and comparing 1D projections of ROOT and McStas data

### Rescale TOF for ROOT
Use metadata of ROOT file to add units and offset to the x-axis

In [ ]:
xaxis_root_rescaled = (xaxis_root * TOF_Time_Channel_Width + TOF_Window_Delay_Register) * tof_tick 

In [ ]:
# Coefficient to rescale 1D McStas projection
mcstas_y_coeff = np.max(data2d_root)/np.max(data2d_mcstas)

In [ ]:
# function to be updated when changing the range of values to calculate the projection
def plots_for_projection_mcstas_root(*project_boundaries):
    """ Plot 2D and 1D projection for user-specified range of y-values"""
    CS_mcstas = ax2[0].contourf(xaxis_mcstas, yaxis_mcstas, data2d_mcstas, cmap=colormap)
    ax2[0].title.set_text('McStas')
    ax2[0].hlines(yaxis_mcstas[project_boundaries[0]], 
                 min(xaxis_mcstas), 
                 max(xaxis_mcstas), 
                 'orange')
    ax2[0].hlines(yaxis_mcstas[project_boundaries[1]], 
                 min(xaxis_mcstas), 
                 max(xaxis_mcstas), 
                 'orange')
    
    CS_root = ax2[1].contourf(xaxis_root_rescaled, yaxis_root, data2d_root.transpose(), cmap=colormap)
    ax2[1].title.set_text('ROOT')
    ax2[1].hlines(yaxis_root[project_boundaries[2]], 
                 min(xaxis_root_rescaled), 
                 max(xaxis_root_rescaled), 
                 'orange')
    ax2[1].hlines(yaxis_root[project_boundaries[3]], 
                 min(xaxis_root_rescaled), 
                 max(xaxis_root_rescaled), 
                 'orange')
    
    # calculate projection
    ax2[2].plot(xaxis_mcstas,
                mcstas_y_coeff*np.sum(data2d_mcstas[project_boundaries[0]:project_boundaries[1],:], axis=0), 
                label='mcstas')
    ax2[2].plot(xaxis_root_rescaled, 
                np.sum(data2d_root.transpose()[project_boundaries[2]:project_boundaries[3],:], axis=0), 
                label='root')
    ax2[2].grid()
    ax2[2].legend()
    
    fig2.colorbar(CS_mcstas, ax=[ax2[0]], location='right')
    fig2.colorbar(CS_root, ax=[ax2[1]], location='right')
    fig2.canvas.draw()
    

# widget to generate 1D projection of mcstas and ROOT data
fig2, ax2 = plt.subplots(3, 1, constrained_layout=True, sharex=True)
fig2.canvas.header_visible = False
plots_for_projection_mcstas_root(*(0, len(yaxis_mcstas)-1, 0, len(yaxis_root)-1))

style_textbox = {'description_width': 'initial'}

def update_mcstas(changes):
    ax2[0].clear()
    ax2[2].clear()
    plots_for_projection_mcstas_root(*changes.new, *set_limits_root.value)
    
def update_root(changes):
    ax2[1].clear()
    ax2[2].clear()
    plots_for_projection_mcstas_root(*set_limits_mcstas.value, *changes.new)

set_limits_mcstas = ipywidgets.IntRangeSlider(
    value=[0, len(yaxis_mcstas)-1],
    min=0, 
    max=len(yaxis_mcstas)-1,
    step=1,
    description='Mac Stas Index boundaries:',
    continuous_update=False,
    readout_format='d',
    style=style_textbox,
    layout = ipywidgets.Layout(width='400px')
)

set_limits_root = ipywidgets.IntRangeSlider(
    value=[0, len(yaxis_root)-1],
    min=0, 
    max=len(yaxis_root)-1,
    step=1,
    description='ROOT Index boundaries:', 
    continuous_update=False,
    readout_format='d',
    style=style_textbox,
    layout = ipywidgets.Layout(width='400px')
)

set_limits_mcstas.observe(update_mcstas, 'value')

set_limits_root.observe(update_root, 'value')

ipywidgets.HBox([set_limits_mcstas, set_limits_root])